In [36]:
## TEST TRT
import cv2
import torch
import random
import time
import numpy as np
import tensorrt as trt

In [37]:
import pycuda.driver as cuda
import pycuda.autoinit

In [76]:
trt_path = '../Weights/best_tiny_400_16.trt'
logger = trt.Logger(trt.Logger.WARNING)
runtime = trt.Runtime(logger)
trt.init_libnvinfer_plugins(logger, "")

with open(trt_path, 'rb') as f:
    engine_data = f.read()

engine = runtime.deserialize_cuda_engine(engine_data)

In [77]:
inputs, outputs, bindings, stream = [], [], [], cuda.Stream()
for binding in engine:
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    dtype = trt.nptype(engine.get_binding_dtype(binding))
    host_mem = cuda.pagelocked_empty(size, dtype)
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    bindings.append(int(device_mem))
    if engine.binding_is_input(binding):
        inputs.append({'host': host_mem, 'device': device_mem})
    else:
        outputs.append({'host': host_mem, 'device': device_mem})

In [78]:
## 입력 이미지 전처리
img = cv2.imread('../test05.jpg')
input_image = cv2.resize(img, (640, 640))
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
input_image = np.transpose(input_image, (2,0,1)).astype(np.float32)/255.0
# input_image = np.ascontiguousarray(input_image.transpose(2, 0, 1))
# input_image = input_image.astype(np.float32) / 255.0

In [79]:
# Copy input data and inference
np.copyto(inputs[0]['host'], input_image.ravel())
cuda.memcpy_htod(inputs[0]['device'], inputs[0]['host'])
context = engine.create_execution_context()
context.execute_v2(bindings = bindings)

True

In [80]:
cuda.memcpy_dtoh(outputs[0]['host'], inputs[0]['device'])

In [81]:
print(outputs)

[{'host': array([998277249], dtype=int32), 'device': <pycuda._driver.DeviceAllocation object at 0x7f288885d0>}, {'host': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [82]:
# Copy Output data and inference
output_data = []
for output in outputs:
    output_data.append(output['host'].reshape(-1))

output_num_detections = output_data[0]
output_boxes = output_data[1].reshape(-1, 4)
output_scores = output_data[2]
output_classes = output_data[3]

In [83]:
print(output_num_detections)

[998277249]


In [85]:
# RESULTS
for box, score, class_id in zip(output_boxes, output_scores, output_classes):
    if score > 0.5:
        x1, y1, x2, y2 = [int(v) for v in box]
        cv2.rectangle(input_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(input_image, f'{class_id}: {score:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12, 2))


cv2.imshow('YOLOv7-tiny trt', input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.7.0) /tmp/pip-install-n_no6scd/opencv-python_73eb20d920614b098d4fd6ad93ce85ca/opencv/modules/core/src/array.cpp:2494: error: (-206:Bad flag (parameter or structure field)) Unrecognized or unsupported array type in function 'cvGetMat'
